In [17]:
from github import Github
from collections import Counter, defaultdict
from pydriller import Repository
import os
import git  
import shutil
import matplotlib.pyplot as plt
import os

## **Achando estatistícas das top 5 linguagens presentes no perfil do dev, seus respectivos repositórios e numero de commits relacionados a cada um.**

In [ ]:
GITHUB_TOKEN = "my token"
USERNAME = "johnatan-si"

g = Github(GITHUB_TOKEN)    
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        repo_name = repo.name
        repo_url = repo.clone_url

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'locAdd': 0,        
                'locRemov': 0,
                'url': repo_url     
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = {lang: reposLanguage[lang] for lang, _ in top_5_languages}

print("\n📊 Top 5 linguagens mais ativas (por número de commits):")
for lang, total_commits in top_5_languages:
    print(f"➡️ {lang}: {total_commits} commits")

print("\n📂 Repositórios por linguagem (Top 5):")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Johnatan 
🔐 Login: johnatan-si
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/8726738?v=4

⚠️ Erro ao acessar SourceLearning-: Git Repository is empty.: 409 {"message": "Git Repository is empty.", "documentation_url": "https://docs.github.com/rest/commits/commits#list-commits", "status": "409"}

📊 Top 5 linguagens mais ativas (por número de commits):
➡️ Java: 246 commits
➡️ Dockerfile: 111 commits
➡️ Python: 81 commits
➡️ Makefile: 59 commits
➡️ AspectJ: 58 commits

📂 Repositórios por linguagem (Top 5):

🗂️ Java:
   - astar0: 2 commits, +0 / -0 LOC
   - AulasPUCMG: 60 commits, +0 / -0 LOC
   - demo-ci: 56 commits, +0 / -0 LOC
   - DownloadGit: 2 commits, +0 / -0 LOC
   - extractgit: 1 commits, +0 / -0 LOC
   - GitBlame: 8 commits, +0 / -0 LOC
   - JExpert: 1 commits, +0 / -0 LOC
   - JExpert2.0: 1 commits, +0 / -0 LOC
   - jreuse: 12 commits, +0 / -0 LOC
   - JSERD2020: 43 commits, +0 / -0 LOC
   - LineCounter: 2 commits, +0 / -0 LOC
   - ufmg.reuso.software: 58 co

In [19]:
output_dir = "graficosCommits"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir)

print("\n📊 Gerando gráficos por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    commits = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        commits.append(repo['commits'])

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, commits, color='skyblue', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"Commits por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Commits")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_commits.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos por linguagem...
✅ Gráfico salvo: graficosCommits/Java_commits.png
✅ Gráfico salvo: graficosCommits/Dockerfile_commits.png
✅ Gráfico salvo: graficosCommits/Python_commits.png
✅ Gráfico salvo: graficosCommits/Makefile_commits.png
✅ Gráfico salvo: graficosCommits/AspectJ_commits.png


## **Clonando repositórios das top 5 linguagens e fazendo contagem de locAdd e locRemov de cada um e gerando graficos respectivos para cada linguagem.**

In [20]:
CLONE_DIR = "gitClones"

g = Github(GITHUB_TOKEN)
user = g.get_user(USERNAME)

print(f"\n📛 Nome: {user.name}")
print(f"🔐 Login: {user.login}")
print(f"🖼️ Foto de perfil: {user.avatar_url}\n")

fotoPerfil = user.avatar_url
languageCommitCounter = Counter()
reposLanguage = defaultdict(list)

for repo in user.get_repos():
    try:
        commits = repo.get_commits()
        commit_count = commits.totalCount
        langs = repo.get_languages()
        clone_url = repo.clone_url
        repo_name = repo.name
        repo_path = os.path.join(CLONE_DIR, repo_name)

        for lang in langs:
            languageCommitCounter[lang] += commit_count
            reposLanguage[lang].append({
                'name': repo_name,
                'commits': commit_count,
                'url': clone_url
            })

    except Exception as e:
        print(f"⚠️ Erro ao acessar {repo.name}: {e}")

top_5_languages = languageCommitCounter.most_common(5)
top5ReposLanguage = defaultdict(list)

print("\n🔁 Clonando repositórios e analisando com PyDriller...")

os.makedirs(CLONE_DIR, exist_ok=True)

for lang, _ in top_5_languages:
    for repo_info in reposLanguage[lang]:
        repo_name = repo_info['name']
        commit_count = repo_info['commits']
        url = repo_info['url']
        local_path = os.path.join(CLONE_DIR, repo_name)

        if not os.path.exists(local_path):
            try:
                print(f"🔽 Clonando {repo_name}...")
                git.Repo.clone_from(url, local_path)
            except Exception as e:
                print(f"❌ Falha ao clonar {repo_name}: {e}")
                continue

        loc_add = 0
        loc_remov = 0

        try:
            for commit in Repository(local_path).traverse_commits():
                for mod in commit.modified_files:
                    loc_add += mod.added_lines
                    loc_remov += mod.deleted_lines
        except Exception as e:
            print(f"❌ Erro analisando {repo_name}: {e}")
            continue

        top5ReposLanguage[lang].append({
            'name': repo_name,
            'commits': commit_count,
            'locAdd': loc_add,
            'locRemov': loc_remov
        })

print("\n📂 Repositórios por linguagem com commits e LOC:")
for lang, repos in top5ReposLanguage.items():
    print(f"\n🗂️ {lang}:")
    for repo in repos:
        print(f"   - {repo['name']}: {repo['commits']} commits, +{repo['locAdd']} / -{repo['locRemov']} LOC")



📛 Nome: Johnatan 
🔐 Login: johnatan-si
🖼️ Foto de perfil: https://avatars.githubusercontent.com/u/8726738?v=4

⚠️ Erro ao acessar SourceLearning-: Git Repository is empty.: 409 {"message": "Git Repository is empty.", "documentation_url": "https://docs.github.com/rest/commits/commits#list-commits", "status": "409"}

🔁 Clonando repositórios e analisando com PyDriller...
🔽 Clonando astar0...
🔽 Clonando AulasPUCMG...
🔽 Clonando demo-ci...
🔽 Clonando DownloadGit...
🔽 Clonando extractgit...
🔽 Clonando GitBlame...
🔽 Clonando JExpert...
🔽 Clonando JExpert2.0...
🔽 Clonando jreuse...
🔽 Clonando JSERD2020...
🔽 Clonando LineCounter...
🔽 Clonando ufmg.reuso.software...
🔽 Clonando F03_API...
🔽 Clonando PUC-GCES-Tutoriais...
🔽 Clonando aula-ci...
🔽 Clonando github_recommend...
🔽 Clonando LinesOfCodeCounter...
🔽 Clonando PUC-GCES-JS...
🔽 Clonando PUC-GCES-PY...

📂 Repositórios por linguagem com commits e LOC:

🗂️ Java:
   - astar0: 2 commits, +1649 / -31 LOC
   - AulasPUCMG: 60 commits, +4922 / -517 

In [21]:
output_dir = "graficosLoc"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir)

print("\n📊 Gerando gráficos de LOC por linguagem...")

for lang, repos in top5ReposLanguage.items():
    nomes_repos = []
    loc_totais = []

    for repo in repos:
        nomes_repos.append(repo['name'])
        total_loc = repo['locAdd'] + repo['locRemov']
        loc_totais.append(total_loc)

    plt.figure(figsize=(10, 6))

    bar_width = 0.4
    x_pos = range(len(nomes_repos))

    bars = plt.bar(x_pos, loc_totais, color='lightcoral', width=bar_width)

    plt.xticks(x_pos, nomes_repos, rotation=45, ha='right')
    plt.xlim(-0.5, len(nomes_repos) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom')

    plt.title(f"LOC por repositório - {lang}")
    plt.xlabel("Repositórios")
    plt.ylabel("Linhas de Código (LOC)")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_loc.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos de LOC por linguagem...
✅ Gráfico salvo: graficosLoc/Java_loc.png
✅ Gráfico salvo: graficosLoc/Dockerfile_loc.png
✅ Gráfico salvo: graficosLoc/Python_loc.png
✅ Gráfico salvo: graficosLoc/Makefile_loc.png
✅ Gráfico salvo: graficosLoc/AspectJ_loc.png


## **Buscando para cada linguagem os top 10 imports feitos.**

In [22]:
import os
import re
from collections import Counter
from pydriller import Repository

LANGUAGE_IMPORT_PATTERNS = {
    'Java': [r'\bimport\s+[\w.*]+;?'],
    'Python': [r'\bimport\s+\w+', r'\bfrom\s+\w+\s+import\s+\w+'],
    'C': [r'#include\s+[<"].+[>"]'],
    'C++': [r'#include\s+[<"].+[>"]', r'\busing\s+namespace\s+\w+'],
    'JavaScript': [r'require\s*\([^)]+\)'],
    'TypeScript': [r'import\s+[^;]+'],
    'PHP': [r'\brequire\s+[^\s;]+', r'\buse\s+[^\s;]+'],
    'Rust': [r'\buse\s+[^\s;]+'],
    'Jupyter Notebook': [r'import\s+\w+', r'from\s+\w+\s+import\s+\w+'],
}


repos_base = "gitClones"
top10ImportsLanguage = {lang: Counter() for lang in top5ReposLanguage.keys()}

repo_to_lang = {}
for lang, repos in top5ReposLanguage.items():
    for repo in repos:
        repo_to_lang[repo['name']] = lang

def extract_imports(text, lang):
    patterns = LANGUAGE_IMPORT_PATTERNS.get(lang, [])
    combined = re.compile('|'.join(patterns))
    matches = combined.findall(text)

    
    if lang == 'Java':
     excluded_prefixes = ('import java.util', 'import Model.', 'import javax.')
     matches = [imp for imp in matches if not imp.strip().startswith(excluded_prefixes)]

    return matches


for repo_name in os.listdir(repos_base):
    repo_path = os.path.join(repos_base, repo_name)

    if not os.path.isdir(repo_path):
        continue

    lang = repo_to_lang.get(repo_name)
    if not lang:
        continue 

    print(f"🔍 Processando repositório: {repo_name} ({lang})")

    try:
        for commit in Repository(repo_path).traverse_commits():

            for file in commit.modified_files:
                code_to_scan = file.source_code or file.diff  
                if code_to_scan:
                    found_imports = extract_imports(code_to_scan, lang)
                    top10ImportsLanguage[lang].update(found_imports)
    except Exception as e:
        print(f"⚠️ Erro ao processar {repo_name}: {e}")
        
for lang in top10ImportsLanguage:
    top10ImportsLanguage[lang] = dict(top10ImportsLanguage[lang].most_common(10))


print("\n📦 Top 10 Imports por Linguagem:")
for lang, imports in top10ImportsLanguage.items():
    print(f"\n🔤 {lang}:")
    if imports:
        for imp, count in imports.items():
            print(f"   - {imp}: {count}")
    else:
        print("   (Nenhum import encontrado)")


🔍 Processando repositório: LineCounter (Java)
🔍 Processando repositório: ufmg.reuso.software (AspectJ)
🔍 Processando repositório: DownloadGit (Java)
🔍 Processando repositório: GitBlame (Java)
⚠️ Erro ao processar GitBlame: SHA b'3a4bff7e794f6965a920f89516e5557ff589fffe' could not be resolved, git returned: b'3a4bff7e794f6965a920f89516e5557ff589fffe missing'
🔍 Processando repositório: aula-ci (Python)
🔍 Processando repositório: extractgit (Java)
⚠️ Erro ao processar extractgit: SHA b'39c696cd940db09f68d0970045d850bde2c0e6df' could not be resolved, git returned: b'39c696cd940db09f68d0970045d850bde2c0e6df missing'
🔍 Processando repositório: JExpert2.0 (Java)
🔍 Processando repositório: PUC-GCES-JS (Python)
🔍 Processando repositório: LinesOfCodeCounter (Python)
🔍 Processando repositório: PUC-GCES-Tutoriais (Dockerfile)
🔍 Processando repositório: demo-ci (Makefile)
🔍 Processando repositório: github_recommend (Makefile)
🔍 Processando repositório: F03_API (Python)
🔍 Processando repositório: JE

In [ ]:
import os
from pydriller import Repository
import git 

def get_user_modified_files(repo_path, github_username):
    """
    Retorna os arquivos modificados por um usuário específico no repositório.
    Args:
        repo_path (str): Caminho local do repositório.
        github_username (str): Nome de usuário do GitHub (ex: 'torvalds').
    Returns:
        set: Arquivos únicos modificados pelo usuário.
    """
    modified_files = set()
    
    try:
        repo = git.Repo(repo_path)

        for commit in Repository(repo_path).traverse_commits():
            author = commit.author
            if (github_username.lower() in author.name.lower() or 
                github_username.lower() in author.email.lower() or 
                (hasattr(author, 'username') and author.username == github_username)):
                
                for file in commit.modified_files:
                    if is_source_file(file.filename):
                        modified_files.add(file.new_path or file.old_path)
    
    except Exception as e:
        print(f"Erro no repositório {repo_path}: {str(e)}")
    
    return modified_files

def is_source_file(filename):
    """Filtra apenas arquivos de código fonte."""
    extensions = ('.py', '.js', '.java', '.c', '.cpp', '.h', '.go', '.rs', '.rb', '.php')
    return filename.endswith(extensions)


user = "johnatan-si"  
git_clones_path = "./gitClones"

for repo_name in os.listdir(git_clones_path):
    repo_path = os.path.join(git_clones_path, repo_name)
    if os.path.isdir(repo_path):
        print(f"\nAnalisando repositório: {repo_name}")
        files = get_user_modified_files(repo_path, user)
        
        if files:
            print(f"Arquivos modificados por {user}:")
            for file in files:
                print(f"  - {file}")
        else:
            print(f"Nenhum arquivo modificado por {user} encontrado.")


Analisando repositório: LineCounter
Nenhum arquivo modificado por johnatan-si encontrado.

Analisando repositório: ufmg.reuso.software
Nenhum arquivo modificado por johnatan-si encontrado.

Analisando repositório: DownloadGit
Nenhum arquivo modificado por johnatan-si encontrado.

Analisando repositório: GitBlame
Arquivos modificados por johnatan-si:
  - GitBlame/src/main/ShellScript2.java
  - GitBlame/src/main/RemoveComments2.java
  - GitBlame/src/main/CopyFIles.java
  - GitBlame/src/main/ConexaoBD.java
  - GitBlame/src/main/ShellScript3.java
  - GitBlame/src/main/Test.java
  - GitBlame/src/main/RemoveComments.java
  - GitBlame/src/main/ShellScript.java
  - GitBlame/src/main/Foo.java
  - GitBlame/src/main/ExecuteShellComand.java
  - GitBlame/src/main/TestaConexao.java
  - GitBlame/src/main/Test2.java
  - GitBlame/src/main/FileUtils.java
  - GitBlame/src/main/CSVReader.java
  - GitBlame/src/main/DataCommit.java
  - GitBlame/src/main/ExecuteShellComand2.java

Analisando repositório: aul

In [39]:
import matplotlib.pyplot as plt
import os
import shutil

output_dir = "graficosImports"

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir)

print("\n📊 Gerando gráficos de imports por linguagem...")

for lang, imports_dict in top10ImportsLanguage.items():
    if not imports_dict:
        print(f"⚠️ Nenhum import encontrado para {lang}, pulando...")
        continue

    import_names = list(imports_dict.keys())
    import_counts = list(imports_dict.values())

    plt.figure(figsize=(12, 6))
    bar_width = 0.5
    x_pos = range(len(import_names))

    bars = plt.bar(x_pos, import_counts, color='mediumpurple', width=bar_width)

    plt.xticks(x_pos, import_names, rotation=45, ha='right')
    plt.xlim(-0.5, len(import_names) - 0.5)

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height,
                 f'{int(height)}',
                 ha='center', va='bottom', fontsize=9)

    plt.title(f"Top 10 Imports - {lang}")
    plt.xlabel("Imports")
    plt.ylabel("Frequência")
    plt.tight_layout()

    filepath = os.path.join(output_dir, f"{lang}_imports.png")
    plt.savefig(filepath)
    plt.close()

    print(f"✅ Gráfico salvo: {filepath}")



📊 Gerando gráficos de imports por linguagem...
✅ Gráfico salvo: graficosImports/Java_imports.png
✅ Gráfico salvo: graficosImports/Dockerfile_imports.png
✅ Gráfico salvo: graficosImports/Python_imports.png
✅ Gráfico salvo: graficosImports/Makefile_imports.png
✅ Gráfico salvo: graficosImports/AspectJ_imports.png


In [40]:
print(languageCommitCounter) #Lingugem e numero de commits
print(top5ReposLanguage) #Top5 linguagens com mais commits e repositórios respectivos com linhas add e remov
print(top10ImportsLanguage) #Top 10 bibliotecas com mais imports por linguagem
print(fotoPerfil)
print("/-" * 40)
print("\n📦 Estrutura detalhada de top5ReposLanguage:\n")

for lang, repos in top5ReposLanguage.items():
    print(f"Linguagem: {lang}")
    for repo in repos:
        print(f"   🔸 Repositório: {repo['name']}")
        print(f"      Quantidade commits (commits): {repo['commits']}")
        print(f"      Linhas adicionadas (locAdd): {repo['locAdd']}")
        print(f"      Linhas removidas (locRemov): {repo['locRemov']}")
    print("-" * 40)


Counter({'Java': 246, 'Dockerfile': 111, 'Python': 81, 'Makefile': 59, 'AspectJ': 58, 'Shell': 52, 'Jupyter Notebook': 7, 'C': 6, 'JavaScript': 4, 'HTML': 4, 'CSS': 4})
defaultdict(<class 'list'>, {'Java': [{'name': 'astar0', 'commits': 2, 'locAdd': 1649, 'locRemov': 31}, {'name': 'AulasPUCMG', 'commits': 60, 'locAdd': 4922, 'locRemov': 517}, {'name': 'demo-ci', 'commits': 56, 'locAdd': 692, 'locRemov': 274}, {'name': 'DownloadGit', 'commits': 2, 'locAdd': 342, 'locRemov': 7}, {'name': 'extractgit', 'commits': 1, 'locAdd': 271, 'locRemov': 0}, {'name': 'GitBlame', 'commits': 8, 'locAdd': 1774, 'locRemov': 789}, {'name': 'JExpert', 'commits': 1, 'locAdd': 1266, 'locRemov': 0}, {'name': 'JExpert2.0', 'commits': 1, 'locAdd': 991, 'locRemov': 0}, {'name': 'jreuse', 'commits': 12, 'locAdd': 2686, 'locRemov': 1397}, {'name': 'JSERD2020', 'commits': 43, 'locAdd': 1141, 'locRemov': 116}, {'name': 'LineCounter', 'commits': 2, 'locAdd': 217, 'locRemov': 0}, {'name': 'ufmg.reuso.software', 'commi